In [1]:
import numpy as np
import matplotlib.pyplot as plt
import PIL
import cairocffi as cairo

In [430]:
def circles_to_surface(circles, height, width):
    surface = cairo.ImageSurface(cairo.FORMAT_RGB24, width, height)
    context = cairo.Context(surface)
    
    context.set_source_rgba(1, 1, 1, 1)
    context.paint()
    
    sorted_circles = circles[circles[:, 2].argsort()]
    
    for circle in sorted_circles:
        context.set_source_rgba(circle[4], circle[5], circle[6], circle[7])
        context.arc(circle[0], circle[1], circle[3], 0, 2 * np.pi)
        context.fill()
    
    return surface

In [82]:
def circles_to_data(circles, height, width):
    return np.ndarray(shape=(height, width, 3), dtype=np.uint8,
                      buffer=circles_to_surface(circles, height, width).get_data())

In [4]:
def save_image(circles, height, width, filename):
    surface = circles_to_surface(circles, height, width)
    surface.write_to_png(filename)

In [384]:
def image_to_data(filename):
    surface = cairo.ImageSurface.create_from_png(filename)
    return np.array(np.ndarray(shape=(surface.get_height(), surface.get_width(), 3), dtype=np.uint8,
                      buffer=surface.get_data()))

In [383]:
a = np.array(image_to_data('mona_lisa.png'))
print(a)

[[[208  98 109]
  [  5 197 127]
  [  0   0 208]
  ...
  [255  98 148]
  [136 255 103]
  [153 141 255]]

 [[101 151 139]
  [255 110 160]
  [148 255  71]
  ...
  [255  30  44]
  [ 72 255  33]
  [ 47  75 255]]

 [[ 14  29  55]
  [255  15  31]
  [ 54 255  12]
  ...
  [255  88 144]
  [139 255  94]
  [150 145 255]]

 ...

 [[ 42  36  61]
  [255  22  16]
  [ 39 255  30]
  ...
  [255  72 181]
  [219 255  96]
  [202 239 255]]

 [[114 213 251]
  [255 121 217]
  [253 255  79]
  ...
  [255  17  51]
  [ 81 255  36]
  [ 70 100 255]]

 [[ 30  64  94]
  [255  55  87]
  [116 255  45]
  ...
  [255  61  90]
  [104 255  39]
  [ 68  82 255]]]


In [439]:
def distance(circles, original_image):
#     print(original_image.shape[0], original_image.shape[1])
#     print(original_image)
#     print((circles_to_data(circles, original_image.shape[0], original_image.shape[1]).astype(np.int16)* 2))
    test_image = circles_to_data(circles, original_image.shape[0], original_image.shape[1])
#     print(original_image - test_image)
    difference = original_image.astype(np.int16) - test_image
#     print(difference)
    return 195075 * original_image.shape[0] * original_image.shape[1] - (difference ** 2).sum()
# zrobic zeby maksymalizowalo
print(distance(np.array([[10, 20, 40, 20, 0.5, 0.1, 0.12, 0.5],
                           [30, 20, 15, 20, 0.5, 0.6, 0.7, 0.8]]), image_to_data('mona_lisa.png')))

83792046602


In [13]:
def test_function(circles, original_image_data):
    results = np.empty(circles.shape[0])
    for i in range(circles.shape[0]):
        results[i] = distance(circles[i, :, :], original_image_data)
    return results
test_function(np.array([[[10, 20, 40, 20, 0.5, 0.1, 0.12, 0.5],
                           [30, 20, 15, 20, 0.5, 0.6, 0.7, 0.8]]]), image_to_data('mona_lisa.png'))

(2, 8)
(2, 8)
(8,)
(8,)
uint64


array([1.39739004e+08])

In [10]:
print(np.array([[[10, 20, 40, 20, 0.5, 0.1, 0.12, 0.5],
                           [30, 20, 15, 20, 0.5, 0.6, 0.7, 0.8]]]).shape)

(1, 2, 8)


In [35]:
circles_to_data(np.array([[10, 20, 40, 20, 0.5, 0.1, 0.12, 0.5],
                           [30, 20, 15, 20, 0.5, 0.6, 0.7, 0.8]]),
                 640, 800)

array([[[240, 221, 156],
        [ 41,  42,  86],
        [  0,   0, 208],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [39]:
print(distance(np.array([[10, 20, 40, 20, 0.5, 0.1, 0.12, 0.5],
                           [30, 20, 15, 20, 0.5, 0.6, 0.7, 0.8]]), image_to_data('mona_lisa.png')))

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
0


In [37]:
image_to_data('mona_lisa.png')

array([[[240, 221, 156],
        [ 41,  42,  86],
        [  0,   0, 208],
        ...,
        [255,  98, 148],
        [136, 255, 103],
        [153, 141, 255]],

       [[101, 151, 139],
        [255, 110, 160],
        [148, 255,  71],
        ...,
        [255,  30,  44],
        [ 72, 255,  33],
        [ 47,  75, 255]],

       [[ 14,  29,  55],
        [255,  15,  31],
        [ 54, 255,  12],
        ...,
        [255,  88, 144],
        [139, 255,  94],
        [150, 145, 255]],

       ...,

       [[ 42,  36,  61],
        [255,  22,  16],
        [ 39, 255,  30],
        ...,
        [255,  72, 181],
        [219, 255,  96],
        [202, 239, 255]],

       [[114, 213, 251],
        [255, 121, 217],
        [253, 255,  79],
        ...,
        [255,  17,  51],
        [ 81, 255,  36],
        [ 70, 100, 255]],

       [[ 30,  64,  94],
        [255,  55,  87],
        [116, 255,  45],
        ...,
        [255,  61,  90],
        [104, 255,  39],
        [ 68,  82, 255]]

In [431]:
save_image(np.array([[10, 20, 40, 20, 0.5, 0.1, 0.12, 0.5],
                           [30, 20, 15, 20, 0.5, 0.6, 0.7, 0.8]]),
                 552, 800, 'b.png')

In [ ]:
image = PIL.Image.open('mona_lisa.jpg').convert('RGBA')
draw = PIL.ImageDraw.Draw(image)
draw.ellipse((0, 0, 150, 250), fill=(255,0,0,28))
print(np.array(image).shape)
image.show()

In [ ]:
def convert(circles):
    fig, ax = plt.subplots()
    sorted_circles = circles[circles[:, 2].argsort()]
    ax.scatter(sorted_circles[:, 0], sorted_circles[:, 1], s=sorted_circles[:, 3],
                        c=sorted_circles[:, 4:])

    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    fig.canvas.draw()
    X = np.array(fig.canvas.renderer._renderer)
    return X

print(convert(np.array([[10, 20, 40, 1000, 0.5, 0.1, 0.12, 0.3], [10.005, 20, 30, 1000, 0.5, 0.6, 0.7, 1]])))

Macież populacji ma wymiar n x k x 8, gdzie n jest liczbą osobników, k liczbą kółek, z których każde jest opisywane przez 8 wielkości:

x, y, z - współrzędne środka, trzeci wymiar jest potrzebny, ponieważ kółka mogą na siebie nachodzić,

r - długość promienia, (2 ** r)

_r, _g, _b - te wartości oznaczają kolor

_alpha - współczynnik intensywności

In [ ]:
def show_image(circles):
    sorted_circles = circles[circles[:, 2].argsort()]
    plt.scatter(sorted_circles[:, 0], sorted_circles[:, 1], s=sorted_circles[:, 3],
                    c=sorted_circles[:, 4:])

    plt.xticks([])
    plt.yticks([])
    plt.show()

In [ ]:
show_image(np.array([[10, 20, 20, 1000, 0.5, 0.1, 0.12, 0.3], [10.005, 20, 30, 1000, 0.5, 0.6, 0.7, 0.9]]))

In [14]:
def mutation_swap_z(circles, probability):
    if np.random.random() > probability:
        return circles
    i = np.random.choice(circles.shape[0])
    j = np.random.choice(circles.shape[0])
    tmp = circles[i, 2]
    circles[i, 2] = circles[j, 2]
    circles[j, 2] = tmp
    return circles

In [486]:
def mutation_random_noise(circles, sigmas, probability):
    noise = np.random.normal(sigmas, size=circles.shape)
    if_mutated = ((np.random.random(size=circles.shape) - probability) < 0).astype(int)
    circles += if_mutated * noise
    return circles

In [499]:
def ES(population_size, number_of_circles, objective_function,
       begin_x, end_x, begin_y, end_y, begin_z, end_z, max_r,
       number_of_offspring, number_of_parents, tau, tau0, number_of_iterations, log_frequency=10):
    
    begin_domain = np.array([begin_x, begin_y, begin_z, 0., 0., 0., 0., 0.])
    end_domain = np.array([end_x, end_y, end_z, max_r, 1., 1., 1., 1.])
    
    best_solution = np.empty((number_of_circles, 8))
    best_solution_objective_value = 0.00

    log_objective_values = np.empty((number_of_iterations, 4))
    log_best_solutions = np.empty((number_of_iterations, number_of_circles, 8))
    log_best_sigmas = np.empty((number_of_iterations, number_of_circles, 8))

    # generating an initial population
    current_population_solutions = (begin_domain + (end_domain - begin_domain)) * \
                                   np.random.rand(population_size, number_of_circles, 8)

    current_population_sigmas = (end_domain - begin_domain) / 2 \
                                * np.ones((population_size, number_of_circles, 8))

    # evaluating the objective function on the current population
    current_population_objective_values = objective_function(current_population_solutions)

    for t in range(number_of_iterations):

        # selecting the parent indices by the roulette wheel method
        fitness_values = current_population_objective_values - current_population_objective_values.min()

        if fitness_values.sum() > 0:
            fitness_values = fitness_values / fitness_values.sum()
        else:
            fitness_values = 1.0 / population_size * np.ones(population_size)
        
        parent_indices = np.random.choice(population_size, (number_of_offspring, number_of_parents),
                                          True, fitness_values).astype(np.int64)

        # creating the children population by Global Intermediere Recombination
        children_population_solutions = np.zeros((number_of_offspring, number_of_circles, 8))
        children_population_sigmas = np.zeros((number_of_offspring, number_of_circles, 8))
        for i in range(number_of_offspring):
            children_population_solutions[i, :] = current_population_solutions[parent_indices[i, :], :].mean(axis=0)
            children_population_sigmas[i, :] = current_population_sigmas[parent_indices[i, :], :].mean(axis=0)
        
        mutation1_probability = 1
        for i in range(children_population_solutions.shape[0]):
            children_population_solutions[i, :] = mutation_random_noise(children_population_solutions[i],
                                                                        children_population_sigmas[i],
                                                                        mutation1_probability)
        
        mutation2_probability = 1
        for i in range(children_population_solutions.shape[0]):
            children_population_solutions[i, :] = mutation_swap_z(children_population_solutions[i],
                                                                  mutation2_probability)
        
        # mutating the children population by adding random gaussian noise 
        # do poprawy
#         children_population_sigmas = children_population_sigmas \
#                                      * np.exp(tau * np.random.randn(number_of_offspring, number_of_circles, 8) \
#                                               + tau_0 * np.random.randn(number_of_offspring, 1, 8))
#         for i in range(number_of_offspring):
#             children_population_solutions[i, :] = children_population_solutions[i, :] \
#                                                   + np.dot(np.random.randn(1, chromosome_length),
#                                                            np.linalg.cholesky(
#                                                                np.tile(children_population_sigmas[i, :],
#                                                                        (chromosome_length, 1))).T)
        
        np.clip(children_population_solutions, begin_domain, end_domain, children_population_solutions)
        
        # evaluating the objective function on the children population
        children_population_objective_values = objective_function(children_population_solutions)
        
        # replacing the current population by (Mu + Lambda) Replacement
        current_population_objective_values = np.hstack([current_population_objective_values,
                                                         children_population_objective_values])
        current_population_solutions = np.vstack([current_population_solutions, children_population_solutions])
        current_population_sigmas = np.vstack([current_population_sigmas, children_population_sigmas])

        I = np.argsort(current_population_objective_values)[::-1]
        current_population_solutions = current_population_solutions[I[:population_size], :]
        current_population_sigmas = current_population_sigmas[I[:population_size], :]
        current_population_objective_values = current_population_objective_values[I[:population_size]]

        # recording some statistics
        if best_solution_objective_value < current_population_objective_values[0]:
            best_solution = current_population_solutions[0, :]
            save_image(best_solution, 552, 800, f'proby/{t}.png')
            best_solution_objective_value = current_population_objective_values[0]
        
        log_objective_values[t, :] = [current_population_objective_values.min(),
                                      current_population_objective_values.max(),
                                      current_population_objective_values.mean(),
                                      current_population_objective_values.std()]
        
        log_best_solutions[t, :] = current_population_solutions[0, :]
        log_best_sigmas[t, :] = current_population_sigmas[0, :]

        if np.mod(t, log_frequency) == 0:
            print("Iteration %04d : best score = %0.8f, mean score = %0.8f." % (t,
                                                                                log_objective_values[:t+1, 1].max(),
                                                                                log_objective_values[t, 2]))

    return best_solution_objective_value, best_solution, log_objective_values, log_best_solutions, log_best_sigmas

In [498]:
best = ES(100, 5, lambda c: test_function(c, image_to_data('mona_lisa.png')),
          0, 800, 0, 552, 0, 80, 1200, 1000, 3, 1, 1, 50, 1)[1] # max_r jest za duze, przykrywa wszystko

Iteration 0000 : best score = 85330259171.00000000, mean score = 83776224399.17999268.
Iteration 0001 : best score = 85330259171.00000000, mean score = 84253707837.97999573.
Iteration 0002 : best score = 85330259171.00000000, mean score = 84445580673.72000122.
Iteration 0003 : best score = 85330259171.00000000, mean score = 84562406031.02000427.
Iteration 0004 : best score = 85330259171.00000000, mean score = 84641531056.94999695.
Iteration 0005 : best score = 85330259171.00000000, mean score = 84673655116.75000000.
Iteration 0006 : best score = 85330259171.00000000, mean score = 84714376918.47999573.
Iteration 0007 : best score = 85330259171.00000000, mean score = 84750373868.82000732.
Iteration 0008 : best score = 85330259171.00000000, mean score = 84807939461.22999573.
Iteration 0009 : best score = 85330259171.00000000, mean score = 84845637985.80999756.
Iteration 0010 : best score = 85330259171.00000000, mean score = 84858644222.77000427.


KeyboardInterrupt: 

In [441]:
save_image(best, 552, 800, 'c.png')

In [421]:
print(best)

[[8.00000000e+02 4.39145379e+02 8.00000000e+01 3.26413186e+01
  1.00000000e+00 5.84128871e-01 7.34575259e-01 6.79783668e-01]
 [4.92930052e+02 5.01001125e+02 5.81428539e+01 3.54771809e+01
  7.85781899e-01 7.70143936e-01 5.90257722e-01 6.91434292e-01]
 [5.06722151e+02 3.53805185e+02 7.10065003e+01 2.63613181e+01
  6.95066761e-01 7.85339787e-01 6.84097481e-01 7.93776545e-01]
 [5.69739214e+02 4.28582722e+02 8.00000000e+01 3.13146806e+01
  1.00000000e+00 8.07548717e-01 8.36287748e-01 7.45707300e-01]
 [4.92472744e+02 4.97076815e+02 6.82884035e+01 3.70000000e+01
  7.92898449e-01 1.00000000e+00 7.10902584e-01 8.11159063e-01]
 [5.81175982e+02 4.52489447e+02 6.34716223e+01 3.12085502e+01
  7.78785380e-01 7.55187506e-01 6.15737094e-01 6.95592328e-01]
 [6.20244977e+02 4.82084994e+02 6.15903305e+01 3.01619383e+01
  6.39658256e-01 1.00000000e+00 6.13570778e-01 1.00000000e+00]
 [5.09149712e+02 3.65708690e+02 5.96570533e+01 3.70000000e+01
  5.58972209e-01 7.35286347e-01 6.79553787e-01 6.78568066e-01]


In [27]:
print(image_to_data('mona_lisa.png'))

[[[ 85 136 129]
  [255  73 124]
  [117 255  74]
  ...
  [255  98 148]
  [136 255 103]
  [153 141 255]]

 [[101 151 139]
  [255 110 160]
  [148 255  71]
  ...
  [255  30  44]
  [ 72 255  33]
  [ 47  75 255]]

 [[ 14  29  55]
  [255  15  31]
  [ 54 255  12]
  ...
  [255  88 144]
  [139 255  94]
  [150 145 255]]

 ...

 [[ 42  36  61]
  [255  22  16]
  [ 39 255  30]
  ...
  [255  72 181]
  [219 255  96]
  [202 239 255]]

 [[114 213 251]
  [255 121 217]
  [253 255  79]
  ...
  [255  17  51]
  [ 81 255  36]
  [ 70 100 255]]

 [[ 30  64  94]
  [255  55  87]
  [116 255  45]
  ...
  [255  61  90]
  [104 255  39]
  [ 68  82 255]]]
